In [4]:
import re #for normalicing texts like titles
from difflib import get_close_matches #fuzzy matching between strings

**metaClean43Brightspace Cleaning**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re #for normalicing texts like titles #Max Code
from pathlib import Path #Max Code

def load_excel(path, filename):
    file = path / filename
    if not file.exists():
        raise FileNotFoundError(f"File not found: {file}")
    return pd.read_excel(file, engine="openpyxl") #openpyxl for efficiency in excel files

base_path = Path(r"C:\Users\dbust\OneDrive\Documentos\Amsterdam_2025\DDBM\Database_Management\Project_DBM") 

#DataFrames: file upload
df     = load_excel(base_path, "metaClean43Brightspace.xlsx")
sales  = load_excel(base_path, "sales_movies.xlsx")
expert = load_excel(base_path, "ExpertReviewsClean43LIWC.xlsx")

#paths to save results
movies_dataframe= base_path / "movies_dataframe"
movies_dict_dataframe= base_path / "movies_dict_dataframe" #Max Code

df.head()
print(df.columns)        # Movie

df_clean=df.drop(('summary'), axis=1) #Delete column
print(df_clean.columns)

#normalizing text
def normalize_columns(df, transformations):
    for col, funcs in transformations.items():  # recorre columnas y funciones
        if col in df.columns:
            for func in funcs:
                if func == "replace":
                    df[col] = df[col].str.replace("| ", "", regex=False)
                else:
                    df[col] = getattr(df[col].str, func)()
    return df
df_clean = normalize_columns(
    df_clean,
    {
        "title": ["strip", "title"],
        "studio": ["strip", "title"],
        "rating": ["strip", "upper", "replace"]  
    }
)
#print(df_clean[["title", "studio", "rating"]].head(10))

ids_url=[]
if 'url' in df_clean.columns:
    id_col_mov= 'url'
    for u in df_clean['url']:
        ids_url.append(u)
#print(ids_url) #urls list
#print(len(ids_url)) #11,364

#movie_id
df_clean = df_clean.sort_values('RelDate') #order of ids from Release Date
df_clean = df_clean.reset_index(drop=True)
df_clean['movie_id'] = range(1, len(df_clean) + 1) #movie_id generation 1...11364

#df_clean[['title', 'RelDate', 'url', 'movie_id']].head(10) #Check
#print(df_clean['movie_id'].is_unique)   # needs to be TRUE

#Funcion that converts a lst if there's sth, if not empty lst
def split_and_strip(s):
    if pd.isna(s) or str(s).strip() == '': #pd.isna(s) checks if cell empty
        return []
    return [item.strip() for item in str(s).split(',') if item.strip() != '']  #separate with coma and clean spaces
   

df_clean['cast'] = df_clean['cast'].apply(split_and_strip) #apply to cast
df_clean['genre'] = df_clean['genre'].apply(split_and_strip) #apply to genre

#print(df_clean.head(10))
#print(df_clean[['title', 'studio', 'rating', 'cast', 'genre']].head(10)) #Check

#url_clean: normalice url in df_clean
#df_clean['url_clean'] = df_clean['url'].fillna('').astype(str).str.strip().str.lower() #normalice urls
#df_clean['url_clean'] = df_clean['url_clean'].replace({'': np.nan}) #recognize empty values as missing data (np.nan)
#print("Movies - urls unique (non-null):", df_clean['url_clean'].nunique(dropna=True)) #counts unique values/ dropna=True: ignores Nan(empty) values
#print(df_clean['url_clean'][:5])

#control duplicates
#dups = df_clean['url_clean'].duplicated(keep=False).sum() #(keep=True(1):Duplicates, keep=False(0):NoDuplicates)
#print("Duplicated url_clean rows in df_clean:", dups)  # Check
#if dups > 0: #show examples of duplicates if they exist
#    print("\nExamples of duplicated urls to analyze:") 
#    mask = df_clean['url_clean'].duplicated(keep=False)      # boolean series
#    dups_df = df_clean[mask]                                 # DF only duplicates
#    subset = dups_df[['title', 'url', 'url_clean', 'movie_id']]  #filter columns to analice
#    print(subset.head(10))                                   # show first 10 lines

#Delete Pending_Dictonary creation url>movie_id (url_to_id)
#source = df_clean.drop_duplicates(subset='url_clean', keep='first') #just chooses first url (no duplicates)
#url_to_id = dict(zip(source['url_clean'], source['movie_id'])) #zip combines url and id, dict converts into dictionary
#print(url_to_id)
#print(f"Dictonary generates with {len(url_to_id)} keys") #debug
#print(list(url_to_id.items())[:5])  #first 5 pairs of key:value

#Strip title from url
#df_clean['slug'] = df_clean['url'].str.split('/').str[-1].str.strip().str.lower()

#save new table in file
#df_clean.to_csv("movies_clean.csv", index=False) #in csv perfect for sql
#df_clean.to_excel("movies_clean.xlsx", index=False) #in excel


Index(['url', 'title', 'studio', 'rating', 'runtime', 'cast', 'director',
       'genre', 'summary', 'awards', 'metascore', 'userscore', 'RelDate'],
      dtype='object')
Index(['url', 'title', 'studio', 'rating', 'runtime', 'cast', 'director',
       'genre', 'awards', 'metascore', 'userscore', 'RelDate'],
      dtype='object')
                                                 url  \
0     https://www.metacritic.com/movie/fantasia-2000   
1  https://www.metacritic.com/movie/lupin-iii-the...   
2       https://www.metacritic.com/movie/next-friday   
3       https://www.metacritic.com/movie/my-dog-skip   
4         https://www.metacritic.com/movie/supernova   
5       https://www.metacritic.com/movie/down-to-you   
6  https://www.metacritic.com/movie/things-you-ca...   
7     https://www.metacritic.com/movie/the-big-tease   
8           https://www.metacritic.com/movie/the-cup   
9          https://www.metacritic.com/movie/santitos   

                                        title       

**sales_movies Cleaning**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

path2=r"C:\Users\dbust\OneDrive\Documentos\Amsterdam_2025\DDBM\Database_Management\Project_DBM\sales_movies.xlsx"
df_sales=pd.read_excel(path2) #df_sales = sales info file

#df_sales.head()
#print(df_sales.columns)  # Sales

df_sales_clean=df_sales.drop(['Unnamed: 8','opening_weekend', 'theatre_count','avg run per theatre', 'creative_type'], axis=1) #Delete columns
#print(df_sales_clean.columns)

df_sales_clean.columns = df_sales_clean.columns.str.strip() #clean column names

#print("Column names:") #to see the names of the columns
#for col in df_sales_clean.columns:
#    print(f"'{col}'")

#creation url sales list (eliminating nule values)
ids_url_sales = df_sales_clean['url'].dropna().tolist()
#print(ids_url_sales[:10]) #first 10 urls on lst
print("Total URLs:", len(ids_url_sales)) #30,612

#sales_id
df_sales_clean = df_sales_clean.sort_values('year', ascending=True) #organice in year order
df_sales_clean = df_sales_clean.reset_index(drop=True) #Reset index
df_sales_clean['sales_id'] = range(1, len(df_sales_clean) + 1) #create sales id column
#print(df_sales_clean[['title', 'year', 'release_date', 'url', 'sales_id']].head(10)) #Check

#df_clean[['title', 'RelDate', 'url', 'movie_id']].head(10) #Check
#print(df_clean['movie_id'].is_unique)   # needs to be TRUE

#url_clean: normalice url in df_clean
#df_sales_clean['url_clean'] = df_sales_clean['url'].fillna('').astype(str).str.strip().str.lower()
#df_sales_clean['url_clean'] = df_sales_clean['url_clean'].replace({'': np.nan})
#print("Sales  - urls unique (non-null):", df_sales_clean['url_clean'].nunique(dropna=True))
#print(df_sales_clean['url_clean'][:5])

#control duplicates
#dups = df_clean['url_clean'].duplicated(keep=False).sum() #(keep=True(1):Duplicates, keep=False(0):NoDuplicates)
#print("Duplicated url_clean rows in df_clean:", dups)  # Check
#if dups > 0: #show examples of duplicates if they exist
#    print("\nExamples of duplicated urls to analyze:") 
#    mask = df_clean['url_clean'].duplicated(keep=False)      # boolean series
#    dups_df = df_clean[mask]                                 # DF only duplicates
#    subset = dups_df[['title', 'url', 'url_clean', 'movie_id']]  #filter columns to analice
#    print(subset.head(10))                                   # show first 10 lines

#Delete pending_Mapping- Dictonary connection url>movie_id (url_to_id)
#df_sales_clean['movie_id'] = df_sales_clean['url_clean'].map(url_to_id) #takes sales url and looks a connection in dic with movie_id & creates movie_id column in sales
#mapped = df_sales_clean['movie_id'].notna().sum() #count how many matches were done
#total = len(df_sales_clean) #total rows in sales
#print(f"Mapped by URL: {mapped} / {total} rows ({mapped/total:.1%})") #percentage of how many matches were made from tot rows in sales
# Debug urls
#print("Ejemplos de url_clean en df_clean:", df_clean['url_clean'].dropna().head(5).tolist())
#print("Ejemplos de url_clean en df_sales_clean:", df_sales_clean['url_clean'].dropna().head(5).tolist())
#common_urls = set(df_clean['url_clean'].dropna()).intersection(df_sales_clean['url_clean'].dropna())
#print(f"Coincidencias de URL entre ambos datasets: {len(common_urls)}")
#print("Ejemplo de coincidencias:", list(common_urls)[:10])


Total URLs: 30612
Sales  - urls unique (non-null): 30612
Duplicated url_clean rows in df_clean: 0
Mapped by URL: 0 / 30612 rows (0.0%)
Ejemplos de url_clean en df_clean: ['https://www.metacritic.com/movie/fantasia-2000', 'https://www.metacritic.com/movie/lupin-iii-the-castle-of-cagliostro', 'https://www.metacritic.com/movie/next-friday', 'https://www.metacritic.com/movie/my-dog-skip', 'https://www.metacritic.com/movie/supernova']
Ejemplos de url_clean en df_sales_clean: ['https://www.the-numbers.com/movie/bakha-satang-(s-korea)', 'https://www.the-numbers.com/movie/looking-for-an-echo', 'https://www.the-numbers.com/movie/kurukshetra', 'https://www.the-numbers.com/movie/little-nicky-(2000)', 'https://www.the-numbers.com/movie/suzhou-river']
Coincidencias de URL entre ambos datasets: 0
Ejemplo de coincidencias: []


**ExpertReviewsClean43LIWC Cleaning**

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#path3=r"C:\Users\dbust\OneDrive\Documentos\Amsterdam_2025\DDBM\Database_Management\Project_DBM\ExpertReviewsClean43LIWC.xlsx"
#df_ExpRev=pd.read_excel(path3) #df_ExpRev = expert review file

#df_ExpRev.head()
#print(df_ExpRev.columns) # Expert Reviews

#df_clean_ExpRev=df_ExpRev.drop(['Analytic', 'Clout', 'Authentic', 'Sixltr', 'Dic', 'function','ppron', 'i', 'we', 'you', 'shehe', 'they', 'ipron',
#       'article', 'prep', 'auxverb', 'negate', 'verb', 'adj',
#       'compare', 'interrog', 'number', 'quant', 'affect', 'female', 'male','insight', 'cause', 'discrep', 'tentat', 'certain', 'differ',
#       'percept', 'see', 'hear', 'feel', 'bio', 'body', 'health', 'sexual',
#       'ingest', 'drives', 'affiliation', 'achieve', 'power', 'reward', 'risk',
#       'focuspast', 'focuspresent', 'focusfuture', 'relativ', 'motion',
#       'space', 'time', 'work', 'leisure', 'home', 'money', 'relig', 'death',
#       'informal', 'swear', 'netspeak', 'assent', 'nonflu', 'filler',
#       'AllPunc', 'Period', 'Comma', 'Colon', 'SemiC', 'QMark', 'Exclam',
#       'Dash', 'Quote', 'Apostro', 'Parenth', 'OtherP', 'conj'], axis=1) #Delete columns
#print(df_clean_ExpRev.columns)